<a href="https://colab.research.google.com/github/shuwang127/SecurityPatch/blob/master/extract_features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preliminary

Import library.

In [0]:
import os
import re
!pip3 install python-Levenshtein
import Levenshtein

Global path.

In [0]:
# global path.
rootPath = './drive/My Drive/Colab Notebooks/'
posPath = rootPath + '/security_patch/'
negPath = rootPath + '/random_commit/'

Keyword definition.

In [0]:
# keyword definition.
MI_keyword = ["overflow", "leak", "buffer", "race", "integer", "null", "dereference", "free", "lock", "byte", \
			  "directory", "bound", "loop", "uninitialized", "stack", "memory","padding", "infinite", "double", \
			  "array", "capture", "pointer", "permission", "size", "length", "division", "crash", "key", "root", "leak"]
# arithmetic operations.
ari_op2 = ["++", "--"]
ari_op1 = ["=", "+", "-", "*", "/", "%"]
# relational operations.
rel_op2 = ["==", "!=", ">=", "<="]
rel_op1 = [">", "<"]
# logical operations.
log_op2 = ["&&", "||"]
log_op1 = ["!", "not", "and", "or"]
# bit operations.
bit_op2 = ["<<", ">>", "bitand", "bitor", "xor"]
bit_op1 = ["~", "&", "|", "^"]
# memory keywords.
mem_keyword = ["malloc", "calloc", "realloc", "free", "memset", "memcmp", "memcpy", "memmove", "sizeof", "new", "delete"]
# if keywords.
if_keyword = ["if", "else", "switch", "case", "&&", "||"]
# loop keywords.
loop_keyword = ["for", "while"]
# jump keywords.
jump_keyword = ["break", "continue", "return", "goto", "throw", "assert"]
# C keywords.
c_keywords = ['auto', 'break', 'case', 'char', 'const', 'continue', 'default', 'do', 'double', 'else', 'enum', 'extern', \
			  'float', 'for', 'goto', 'if', 'inline', 'int', 'long', 'register', 'restrict', 'return', 'short', 'signed', \
			  'sizeof', 'static', 'struct', 'switch', 'typedef', 'union', 'unsigned', 'void', 'volatile', 'while', 'bool', \
			  'alignas', 'alignof', 'bool', 'complex', 'imaginary', 'noreturn', 'static_assert', 'thread_local']
# C++ keywords.
cpp_keywords = ['alignas', 'alignof', 'and', 'and_eq', 'asm', 'atomic_cancel', 'atomic_commit', 'atomic_noexcept', 'auto', \
				'bitand', 'bitor', 'bool', 'break', 'case', 'catch', 'char', 'char8_t', 'char16_t', 'char32_t', 'class', \
				'compl', 'concept', 'const', 'consteval', 'constexpr', 'const_cast', 'continue', 'co_await', 'co_return', \
				'co_yield', 'decltype', 'default', 'delete', 'do', 'double', 'dynamic_cast', 'else', 'enum', 'explicit', \
				'export', 'extern', 'false', 'float', 'for', 'friend', 'goto', 'if', 'import', 'inline', 'int', 'long', \
				'module', 'mutable', 'namespace', 'new', 'noexcept', 'not', 'not_eq', 'nullptr', 'operator', 'or', 'or_eq', \
				'private', 'protected', 'public', 'reflexpr', 'register', 'reinterpret_cast', 'requires', 'return', 'short', \
				'signed', 'sizeof', 'static', 'static_assert', 'static_cast', 'struct', 'switch', 'synchronized', 'template', \
				'this', 'thread_local', 'throw', 'true', 'try', 'typedef', 'typeid', 'typename', 'union', 'unsigned', 'using', \
				'virtual', 'void', 'volatile', 'wchar_t', 'while', 'xor', 'xor_eq', 'printf']
# dictionary keywords.
dir_keyword = ["\'\'", "\'.\'", "\'..\'", "\'/../\'", "\'../\'", "\'/..\'", "\'/\'", "\'\\\'", "\'/\\\'",\
				'\"\"', '\".\"', '\"..\"', '\"/../\"', '\"../\"', '\"/..\"', '\"/\"', '\"\\\"', "\'/\\\'"]
# race keywords.
race_keyword = ["release", "lock", "mutex", "unlock"]
# not keywords.
not_keyword = ["==0", "!=0", "==null", "!=null", "!"]

Variable definition.

In [0]:
# define cache.
name = []
label = []
MI = []
# diff, hunk, func number.
diff_num = []
hunk_num = []
func_num = []
# line number.
line_num_total = []
line_num_net = []
line_num_del = []
line_num_add = []
# character number.
char_num_total = []
char_num_net = []
char_num_del = []
char_num_add = []
# memory number.
mem_num_total = []
mem_num_net = []
mem_num_del = []
mem_num_add = []
# if number.
if_num_total = []
if_num_net = []
if_num_del = []
if_num_add = []
# jump number.
jump_num_total = []
jump_num_net = []
jump_num_del = []
jump_num_add = []
# loop number.
loop_num_total = []
loop_num_net = []
loop_num_del = []
loop_num_add = []
# arithmetic number.
ari_num_total = []
ari_num_net = []
ari_num_del = []
ari_num_add = []
# relational number.
rel_num_total = []
rel_num_net = []
rel_num_del = []
rel_num_add = []
# logical number.
log_num_total = []
log_num_net = []
log_num_del = []
log_num_add = []
# bit number.
bit_num_total = []
bit_num_net = []
bit_num_del = []
bit_num_add = []
# call number.
call_num_total = []
call_num_net = []
call_num_del = []
call_num_add = []
# variable number.
var_num_total = []
var_num_net = []
var_num_del = []
var_num_add = []
# global
global_sim = []
global_norm_sim = []
# dictionary number.
dir_num_total = []
dir_num_net = []
dir_num_del = []
dir_num_add = []
# others.
cap_num = []
race_num = []
not_zero = []
preprocess = []

# main function

main function